In [108]:
import pandas as pd
import numpy as np
import datetime
from pytz import timezone

In [109]:
data_dir = r"./transport/"

In [124]:
transport = pd.read_csv(data_dir + r"transport.csv", encoding = "ISO-8859-1", sep=";")
temperatures = pd.read_csv(data_dir + r"temperatures.csv", encoding = "ISO-8859-1", sep=";")

### 1. Transport

#### a. Inspect data

In [125]:
transport.shape
# We have data on 3790 transports

(3790, 16)

In [126]:
# Show first 3 rows, 4 random samples, last 3 rows
pd.concat([transport.head(3), 
           transport.sample(8).sort_values(by='Transport ID'), 
           transport.tail(3)])

,Transport ID,Container ID,Container Type,Forwarder,Trucking Time Start,Ground Handler 1,Airport 1,Ground Handler 2,Airport 2,Ground Handler 3,Airport 3,Ground Handler 4,Airport 4,Ground Handler 5,Airport 5,Trucking Time End
0,5559,000-10081,770C,NaN,24.07.2013,NaN,NaN,NaN,NaN,##,##,##,##,##,##,24.07.2013
1,5560,000-10081,770C,NaN,26.11.2013,NaN,NaN,NaN,NaN,##,##,##,##,##,##,26.11.2013
2,5561,000-10080,770C,NaN,24.07.2013,NaN,NaN,NaN,NaN,##,##,##,##,##,##,24.07.2013
1270,7229,000-10349,770C,Bischof,23.11.2015 09:00 +0100,NaN,NaN,NaN,NaN,##,##,##,##,##,##,NaN
1732,7598,011-10050,1500C,Bischof,25.05.2016 08:00 +0200,NaN,NaN,NaN,NaN,##,##,##,##,##,##,NaN
2574,8271,002-10000,770CRT,Frigo Trans Fussgoenheim,13.02.2017 14:00 +0100,NaN,NaN,NaN,NaN,##,##,##,##,##,##,14.02.2017 08:00 +0100
2862,8463,000-10353,770C,Jet Trans Innsbruck,08.06.2017 15:00 +0200,NaN,NaN,NaN,NaN,##,##,##,##,##,##,NaN
3264,8830,011-10114,1500C,Emirates SkyCargo,08.10.2017 07:00 +0400,NaN,NaN,NaN,NaN,##,##,##,##,##,##,08.10.2017 09:00 +0400
3300,8887,032-10002,32,Courier Factory,19.10.2017 15:00 +0200,NaN,NaN,NaN,NaN,##,##,##,##,##,##,20.10.2017 09:00 +0200
3332,8927,002-10011,770CRT,DHL Global Forwarding,03.11.2017 15:00 +0100,NaN,NaN,NaN,NaN,##,##,##,##,##,##,05.11.2017 05:00 +0100


Issues:
- Trucking time date format looks good (assume UTC is implied) 
- Initially: no time recorded. Add a flag to indicate whether time was included (otherwise the time estimates from start to end will suggest a precision that is not there)
- "##"'s to indicate missing values (replace this)

#### b. Clean-up data

_ Replace ## with np.nan's _

In [127]:
## Replace "  ##" with np.nan's
transport.replace(" ##", np.nan, inplace=True)

_ Convert datetime's _

In [128]:
## Noodling around how to Convert times to datetime objects
transp_dt_10 = transport['Trucking Time Start'][10]
transp_dt_1000 = transport['Trucking Time Start'][1000]
print(transp_dt_10)
print(transp_dt_1000)
transp_dt_10_datetime = pd.to_datetime(transp_dt_10)
transp_dt_1000_datetime = pd.to_datetime(transp_dt_1000)
print('-----------------------')
print(transp_dt_10_datetime)
print(transp_dt_1000_datetime)

26.11.2013
15.07.2015 07:00 +0200
-----------------------
2013-11-26 00:00:00
2015-07-15 05:00:00


In [129]:
## Noodling around: how to make the flag
# Different options: on length, splitting
# Chosen route for now: split on colon. Time should always be indicated by a colon, and will result in a 
# split of at least two elements
print(transp_dt_10.split(':'))
print(transp_dt_1000.split(':'))

['26.11.2013']
['15.07.2015 07', '00 +0200']


In [130]:
transport['full_timestamp_flag'] = transport['Trucking Time Start'].apply(lambda x: len(x.split(':')) >1)

In [131]:
## Conversion to datetime, UTC seems to work out-of-the-box 
transport['Trucking Time Start'] = pd.to_datetime(transport['Trucking Time Start'])
transport['Trucking Time End'] = pd.to_datetime(transport['Trucking Time End'])

_ Inspect converted DataFrame _

In [132]:
pd.concat([transport.head(3), 
           transport.sample(8).sort_values(by='Transport ID'), 
           transport.tail(3)])

,Transport ID,Container ID,Container Type,Forwarder,Trucking Time Start,Ground Handler 1,Airport 1,Ground Handler 2,Airport 2,Ground Handler 3,Airport 3,Ground Handler 4,Airport 4,Ground Handler 5,Airport 5,Trucking Time End,full_timestamp_flag
0,5559,000-10081,770C,NaN,2013-07-24 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-07-24 00:00:00,False
1,5560,000-10081,770C,NaN,2013-11-26 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-11-26 00:00:00,False
2,5561,000-10080,770C,NaN,2013-07-24 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-07-24 00:00:00,False
49,5608,000-10485,770C,NaN,2014-01-17 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-01-17 00:00:00,False
961,6830,000-10215,770C,Jet Trans Innsbruck,2015-04-06 00:00:00,NaN,FRA,NaN,MVD,NaN,NaN,NaN,NaN,NaN,NaN,NaT,False
1346,7335,000-10325,770C,Bischof,2016-01-18 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-20 11:00:00,True
1410,7365,011-10011,1500C,Bischof,2016-02-15 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,True
1508,7412,011-10040,1500C,Bischof,2016-02-03 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-03 18:00:00,True
2276,7996,012-10010,1500CRT,1524,2016-10-19 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-10-21 08:00:00,True
2467,8169,011-10058,1500C,Bischof,2016-05-12 07:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-05-12 09:00:00,True


### Inspect temperatures

In [67]:
# Show first 3 rows, 4 random samples, last 3 rows
pd.concat([temperatures.head(3), temperatures.sample(6), temperatures.tail(3)])

,Transport ID,Time,Temperature (Internal),Temperature (External)
0,5559,NaN,NaN,NaN
1,5560,NaN,NaN,NaN
2,5561,NaN,NaN,NaN
142315,8745,Sun Sep 17 01:44:15 CEST 2017,19.1,18.9
162853,8998,Tue Oct 31 17:09:19 CET 2017,26.9,27.5
132855,8682,Wed Sep 06 04:18:43 CEST 2017,21.6,22.0
38457,7656,Sat Jun 04 16:32:38 CEST 2016,4.1,5.9
80391,8307,Wed Mar 01 07:35:45 CET 2017,19.5,18.2
1669,6458,Fri Aug 22 13:34:12 CEST 2014,5.8,19.1
233358,9351,Tue Mar 06 09:32:07 CET 2018,16.6,15.4


Issues:
- It seems that Transport ID is shown only once, after which the timeseries follows
- Mix of Central European Time and Central European Summer Time 

https://www.timeanddate.com/time/zones/cet

NB: CET = UTC + 1. CEST = UTC + 2

In [74]:
temperatures.Time[2000]

'Thu Sep 18 20:26:09 CEST 2014'

In [ ]:
fmt = "%Y-%m-%d %H:%M:%S %Z%z"

In [68]:
# fill the NaN's with the previous values, convert to int
temperatures['Transport ID'] = temperatures['Transport ID'].fillna(method='ffill').astype('int')

In [69]:
pd.concat([temperatures.head(3), temperatures.sample(4), temperatures.tail(3)])

,Transport ID,Time,Temperature (Internal),Temperature (External)
0,5559,NaN,NaN,NaN
1,5560,NaN,NaN,NaN
2,5561,NaN,NaN,NaN
221017,9281,Mon Feb 19 11:46:44 CET 2018,23.9,21.9
182125,9106,Thu Dec 14 23:13:27 CET 2017,12.1,11.1
201705,9203,Sun Nov 26 07:35:58 CET 2017,17.4,15.2
43015,7752,Sun Jul 24 05:11:51 CEST 2016,20.6,20.5
233358,9351,Tue Mar 06 09:32:07 CET 2018,16.6,15.4
233359,9351,Tue Mar 06 09:42:07 CET 2018,16.7,16.1
233360,9351,Tue Mar 06 09:52:07 CET 2018,16.7,NaN


 ### <font color='red'> NB: verify that forward-fill is indeed the correct approach </font>

In [62]:
# Let's not join here. Put into DB instead. 
# df = transport.join(other=temperatures.set_index('Transport ID'), on = 'Transport ID')

In [66]:
temperatures.tail(100)

,Transport ID,Time,Temperature (Internal),Temperature (External)
233261,9351,Mon Mar 05 17:22:07 CET 2018,18.7,16.4
233262,9351,Mon Mar 05 17:32:07 CET 2018,18.6,16.2
233263,9351,Mon Mar 05 17:42:07 CET 2018,18.6,16.1
233264,9351,Mon Mar 05 17:52:07 CET 2018,18.6,15.9
233265,9351,Mon Mar 05 18:02:07 CET 2018,18.6,15.8
233266,9351,Mon Mar 05 18:12:07 CET 2018,18.6,15.7
233267,9351,Mon Mar 05 18:22:07 CET 2018,18.5,15.6
233268,9351,Mon Mar 05 18:32:07 CET 2018,18.5,15.5
233269,9351,Mon Mar 05 18:42:07 CET 2018,18.5,15.4
233270,9351,Mon Mar 05 18:52:07 CET 2018,18.5,15.3
